In [11]:
NUM_NEGATIVE_SAMPLES=20

from random import randint
def N_random_values_in_list(full_list, N):
    x=0
    lower_bound  = 0
    upper_bound = len(full_list)-1
    sample_list=[]
    random_nums=[]
    while x < min(N,len(full_list)):
        random_num = randint(lower_bound, upper_bound) # inclusive range
        if random_num in random_nums:
            continue
        else:
            random_nums.append(random_num)
            x += 1
    return [full_list[i] for i in random_nums]

In [12]:
import torch
import gzip
import numpy as np

def convert_to_list(filename):
    if filename.endswith('gz'):
        with gzip.open(filename,'r')as f:
            text_tokens = f.readlines()
    else:
        with open(filename, 'r') as f:
            text_tokens = f.readlines()
    text_tokens = [token.replace('\n','').split('\t') for token in text_tokens]
    text_tokens = [[token[0], token[1].split(' '), token[2].split(' ')] for token in text_tokens]
                   
    return text_tokens

#Sample:question_id, similar_question_id, negative_question_id
def convert_to_samples(filename):
    my_list=convert_to_list(filename)
    new_samples=[]
    for original_sample in my_list:
        for similar in original_sample[1]:
            random_negative_samples = N_random_values_in_list(original_sample[2],NUM_NEGATIVE_SAMPLES)
            new_samples.append([original_sample[0], similar, random_negative_samples])# change this to include all negative 
                                                                                     # examples later
    return new_samples
def make_lookup_table_for_training_data(filename):
    lookup={}
    text_token_list=convert_to_list(filename)
    for token in text_token_list:
        lookup[token[0]] = {'title':token[1],'question':token[2]}
    return lookup
        
#takes  sample_ids of [[q1,p1,n1],[q2,p2,n2]....]
#outputs titles like [[q1_title, p1_title, n1_title],[q2_title,p2_title,n2_title]...]
def convert_sampleids_to_titles(sample_ids,lookup):
    #each sample_id [question_id, pos_id, [neg_ids]]
    #print type(sample_ids)==list, "first"
   
    titles = []
    for sample_id in sample_ids:
        
         #flatten list: [question_id, pos_id, [neg_ids]] --> [question_id, pos_id, neg_id1, neg_id2, ...]
        sample_id= sample_id[:2]+sample_id[2][:]
        #sample_id : question_id, similar_question_id, negative_question_id
        try:
            titles.append([lookup[str(identity)]['title'] for identity in sample_id])
           # print type(sample_id)==list
        except:
            print sample_id, "is sample id", type(sample_id)==list
    return titles
    

In [13]:
#text_tokenized.txt.gz has id \t title \t question body
text_tokenized='askubuntu/text_tokenized.txt.gz'

#train_random.txt
#(1) the query question ID, (2) the list of similar question IDs, and (3) the list of randomly selected question IDs.
train_random_filename='askubuntu/train_random.txt'

#Each line contains (1) the query question ID, (2) the list of similar question IDs, (3) the list of 20 candidate question IDs and (4) the associated BM25 scores of these questions computed by the Lucene search engine. The second field (the set of similar questions) is a subset of the third field.
dev_filename='askubuntu/dev.txt'
test_filename='askubuntu/test.txt'

train_samples = convert_to_samples(train_random_filename)
dev_samples = convert_to_samples(dev_filename)
test_samples = convert_to_samples(test_filename)

lookup = make_lookup_table_for_training_data(text_tokenized)
train_list = convert_to_list(train_random_filename)
train_titles_only = convert_sampleids_to_titles(train_samples, lookup)

In [14]:
def remove_non_ascii(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

word_embeddings = 'askubuntu/vector/vectors_pruned.200.txt.gz'
f = gzip.open(word_embeddings, 'r')
wv_text = [ ]
lines = f.readlines()
for line in lines:
    wv_text.append(line.strip())

word_to_vec = {}

for line in wv_text:
    parts = line.split()
    word = parts[0]
    vector = np.array([float(v) for v in parts[1:]])
    word_to_vec[word] = vector
f.close()

def extract_features(word):
    try:
        word=remove_non_ascii(word)
        word=word.encode('utf-8')
    except:
        print(word)
    return word_to_vec.get(word,[0.0 for i in range(200)])

In [15]:
def find_maximum_title_and_body_length(lookup_table):
    max_len_title = -1
    max_len_question = -1
    max_len_question_id = 0
    for key, dict_val in lookup_table.iteritems():
        len_title = len(dict_val['title'])
        len_question = len(dict_val['question'])
        if len_title > max_len_title:
             max_len_title = len_title
        if len_question > max_len_question:
            max_len_question = len_question
            max_len_question_id = key
    return max_len_title, max_len_question

def title_to_feature_matrix(title_word_list):
    feature_matrix = []
    for word in title_word_list:
        word_features = extract_features(word)
        feature_matrix.append(word_features)
    #Pad the feature with zeros to ensure all inputs to the net have the same dimension
    feature_matrix += [[0.] * NUM_FEATURES_PER_WORD] * (MAX_TITLE_LENGTH - len(title_word_list))
    #print np.array(feature_matrix).T.shape
    return np.array(feature_matrix).T

In [16]:
MAX_TITLE_LENGTH, MAX_BODY_LENGTH = find_maximum_title_and_body_length(lookup)
NUM_FEATURES_PER_WORD = 200
INPUT_DIM = (MAX_TITLE_LENGTH, NUM_FEATURES_PER_WORD)

In [21]:
import torch.utils.data as data_utils
BATCH_SIZE=50

#for each tuple of titles make a feature vector that is num_titles x 200 x 38
# where num_titles = 1 (target) + 1 (positive) + n (negative) 
features = []
for i in range(len(train_titles_only[:1000])):   # we should include all but this is just for simplicity 
    sample=train_titles_only[i]
    if i%1000==0:
        print i
    target_title = sample[0]
    positive_title = sample[1]
    negative_titles = sample[2:]
        
    target_features = title_to_feature_matrix(target_title)
    positive_features = title_to_feature_matrix(positive_title)
    n_negative_features = [title_to_feature_matrix(negative_title) for negative_title in negative_titles]
    
    #all features should be size num_titles x feature size ....
    # where num_titles = 1 (target) + 1 (positive) + n (negative) 
    all_features=[[[0]*MAX_TITLE_LENGTH] * NUM_FEATURES_PER_WORD]*(2+NUM_NEGATIVE_SAMPLES)

    all_features[0]=target_features
    all_features[1]=positive_features
    all_features[2:len(n_negative_features)+2]=n_negative_features
    features.append(all_features)
print("created features", len(all_features))

print torch.FloatTensor(features).size(0)
targets = torch.LongTensor(torch.FloatTensor(features).size(0), 1).zero_()
train = data_utils.TensorDataset(torch.FloatTensor(features), targets)
print("created features")
train_loader = data_utils.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

0
('created features', 22)
1000
created features


In [ ]:
len(features[1])

In [9]:
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable


class CNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CNN, self).__init__()
        
        self.conv1d = nn.Conv1d(200, 200, KERNEL_SIZE)
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(200, hidden_dim, KERNEL_SIZE),
            nn.ReLU(),
            nn.AvgPool1d(KERNEL_SIZE)
        )
    def forward(self, x):
        x = self.layer1(x)
        return x

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
KERNEL_SIZE = 3
INPUT_SIZE = 200
HIDDEN_SIZE = 600
LEARNING_RATE = 1e-3
NUM_EPOCHS = 25
net = CNN(INPUT_SIZE, HIDDEN_SIZE)

criterion = nn.MultiMarginLoss(p=1, margin=0.2, weight=None, size_average=True) #HAHA just put these in to look smart 
optimizer = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)

print "going in to training"
# ----TRAINING
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
   # for sample in features:
    for i,(sample,label) in enumerate(train_loader):
        if i%25==0:
            print i
        target_features=Variable(sample[:,0])
        positive_features=Variable(sample[:,1])
        n_negative_features=[Variable(sample[:,i]) for i in range(2, len(sample[0]))]

        target_matrix = net(target_features)
        positive_matrix = net(positive_features)
        n_negative_matrices = [net(negative_features) for negative_features in n_negative_features]
                
        target_vec = target_matrix.data.numpy()[0].reshape(50,1,-1)
        positive_vec = positive_matrix.data.numpy()[0].reshape(50,1,-1)
        n_negative_vecs = [negative_matrix.data.numpy()[0].reshape(50,1,-1) for negative_matrix in n_negative_matrices]
        #print target_vec, "is target_vec"
        cos_sim_positive = [cosine_similarity(target_vec[i], positive_vec[i]) for i in range(BATCH_SIZE)]
        all_cos_sim_negative=[]
        for i in range(BATCH_SIZE):
            batch_of_cos_sims=[]
            for j in range(len(n_negative_vecs)):
                negative_vec=n_negative_vecs[j]
                #print negative_vec[0], "is negative_vec", i
                batch_of_cos_sims.append(cosine_similarity(target_vec[i], negative_vec[i]))
            all_cos_sim_negative.append(batch_of_cos_sims)

        all_cos_sim_negative=np.array(all_cos_sim_negative)[:,:,:,0]
        cos_sim_positive = np.array([cos_sim_positive[i] for i in range(BATCH_SIZE)])
        # cos sims should be BATCH_SIZE x num questions (i.e. 2 when only using 1 negative example or 21 when using 20)        
        cos_sims=np.concatenate((cos_sim_positive, all_cos_sim_negative),axis=1)
        # print cos_sims[0], "is cos_sims for first in batch"
        max_idx = Variable(torch.FloatTensor(np.array(cos_sims)), requires_grad=True) #use axis = 1 when we use more negative examples later
        max_idx=max_idx[:,:,0]
       # y = Variable(torch.LongTensor([0]),requires_grad=True)
        y = Variable(label)
        y=y[:,0]
        """
        because we know the 0th index in cos_sims is always the example we expect to most closely match 
        the target question
        """

        optimizer.zero_grad()
        
        loss = criterion(max_idx, y)
        print loss, max_idx, "is loss and max idx"
        loss.backward()
    
        running_loss += loss.data[0]
#         print running_loss
        optimizer.step()

    print "Loss after epoch " + str(epoch) + " :" + str(running_loss)
# ----END TRAINING



going in to training
0
Variable containing:
 0.1816
[torch.FloatTensor of size 1]
 Variable containing:
 0.8883  0.8970  0.8900  ...   0.8951  0.8605  0.9097
 0.9376  0.9077  0.8832  ...   0.8929  0.8683  0.8914
 0.9151  0.9202  0.8916  ...   0.8805  0.8903  0.9387
          ...             ⋱             ...          
 0.9234  0.9189  0.8779  ...   0.8753  0.9551  0.9537
 0.7997  0.8528  0.8425  ...   0.8568  0.8140  0.8937
 0.8752  0.9230  0.9003  ...   0.9164  0.9191  0.9005
[torch.FloatTensor of size 50x21]
 is loss and max idx
Variable containing:
 0.1594
[torch.FloatTensor of size 1]
 Variable containing:
 0.8341  0.8782  0.8694  ...   0.8399  0.8560  0.8582
 0.8803  0.9227  0.8617  ...   0.8947  0.8915  0.8477
 0.9140  0.8758  0.9291  ...   0.8467  0.8440  0.8303
          ...             ⋱             ...          
 0.9608  0.9473  0.9378  ...   0.9127  0.9406  0.9057
 0.8899  0.8773  0.8612  ...   0.8416  0.8610  0.7696
 0.9032  0.8909  0.8996  ...   0.9146  0.8970  0.8942
[tor

Variable containing:
 0.2055
[torch.FloatTensor of size 1]
 Variable containing:
 0.8481  0.9175  0.8832  ...   0.8213  0.8986  0.9425
 0.8811  0.9458  0.9153  ...   0.9052  0.9414  0.9259
 0.8787  0.9112  0.8824  ...   0.8297  0.8436  0.8542
          ...             ⋱             ...          
 0.9342  0.9421  0.9668  ...   0.9022  0.9283  0.9413
 0.7753  0.8041  0.8732  ...   0.7856  0.8463  0.8376
 0.8918  0.9581  0.9420  ...   0.8680  0.9379  0.9540
[torch.FloatTensor of size 50x21]
 is loss and max idx
Variable containing:
 0.1708
[torch.FloatTensor of size 1]
 Variable containing:
 0.8394  0.8728  0.8470  ...   0.8280  0.8963  0.8718
 0.8818  0.9226  0.8918  ...   0.8449  0.9070  0.9188
 0.8996  0.8562  0.8924  ...   0.8925  0.8920  0.9096
          ...             ⋱             ...          
 0.9157  0.9115  0.9489  ...   0.9501  0.9059  0.9378
 0.9047  0.8650  0.8389  ...   0.7901  0.7524  0.8269
 0.8685  0.8903  0.9212  ...   0.8441  0.8648  0.8542
[torch.FloatTensor of size 

Variable containing:
 0.1661
[torch.FloatTensor of size 1]
 Variable containing:
 0.9571  0.9122  0.9231  ...   0.9515  0.8609  0.8994
 0.9376  0.9335  0.9663  ...   0.9399  0.8698  0.9187
 0.8896  0.8468  0.8590  ...   0.8686  0.8296  0.8953
          ...             ⋱             ...          
 0.9274  0.9214  0.9168  ...   0.9353  0.8627  0.8897
 0.9039  0.8633  0.8189  ...   0.8424  0.8149  0.9019
 0.9030  0.9279  0.8879  ...   0.8825  0.8363  0.9251
[torch.FloatTensor of size 50x21]
 is loss and max idx
Variable containing:
1.00000e-02 *
  9.7488
[torch.FloatTensor of size 1]
 Variable containing:
 0.9910  0.8234  0.9050  ...   0.8483  0.9043  0.9069
 0.9945  0.7885  0.9182  ...   0.7796  0.9299  0.9587
 0.9874  0.7977  0.9001  ...   0.8216  0.9271  0.9120
          ...             ⋱             ...          
 0.9951  0.8819  0.9513  ...   0.8702  0.9301  0.9654
 0.9899  0.8099  0.8640  ...   0.7127  0.8684  0.8696
 0.9939  0.8726  0.9524  ...   0.8458  0.9158  0.9451
[torch.Float

Variable containing:
 0.2128
[torch.FloatTensor of size 1]
 Variable containing:
 0.8705  0.8745  0.8709  ...   0.8889  0.9013  0.7939
 0.9027  0.8908  0.8886  ...   0.8837  0.9069  0.8917
 0.8317  0.9044  0.8646  ...   0.8266  0.8818  0.7781
          ...             ⋱             ...          
 0.8988  0.9278  0.9430  ...   0.9188  0.9213  0.9198
 0.8141  0.8338  0.8850  ...   0.8622  0.8431  0.8241
 0.8657  0.9144  0.9166  ...   0.9296  0.9204  0.8785
[torch.FloatTensor of size 50x21]
 is loss and max idx
Variable containing:
 0.2661
[torch.FloatTensor of size 1]
 Variable containing:
 0.8602  0.9652  0.8753  ...   0.9616  0.8722  0.9170
 0.8057  0.9495  0.9134  ...   0.9409  0.8714  0.9343
 0.8206  0.9514  0.8294  ...   0.9007  0.8500  0.9306
          ...             ⋱             ...          
 0.8805  0.9833  0.9283  ...   0.9594  0.9368  0.9549
 0.7310  0.9386  0.8734  ...   0.9350  0.7688  0.7590
 0.8284  0.9372  0.8851  ...   0.9069  0.8510  0.9187
[torch.FloatTensor of size 

Variable containing:
 0.2003
[torch.FloatTensor of size 1]
 Variable containing:
 0.9354  0.9128  0.9173  ...   0.9088  0.9266  0.9364
 0.9065  0.9074  0.9134  ...   0.9270  0.9368  0.8803
 0.8917  0.8877  0.9391  ...   0.9272  0.8999  0.9023
          ...             ⋱             ...          
 0.9634  0.9682  0.9629  ...   0.9698  0.9517  0.9604
 0.8054  0.7871  0.8105  ...   0.8415  0.8585  0.8144
 0.8834  0.8924  0.9409  ...   0.9528  0.9483  0.9325
[torch.FloatTensor of size 50x21]
 is loss and max idx
Variable containing:
 0.1582
[torch.FloatTensor of size 1]
 Variable containing:
 0.9592  0.9488  0.8754  ...   0.9133  0.9110  0.9583
 0.9020  0.8972  0.8605  ...   0.9264  0.8941  0.8823
 0.9167  0.8908  0.8833  ...   0.8951  0.8633  0.8775
          ...             ⋱             ...          
 0.9218  0.9182  0.9589  ...   0.9128  0.8999  0.9455
 0.8965  0.8817  0.9045  ...   0.8926  0.8407  0.9212
 0.9377  0.9089  0.8677  ...   0.9102  0.9310  0.9100
[torch.FloatTensor of size 

...

Variable containing:
 0.1639
[torch.FloatTensor of size 1]
 Variable containing:
 0.9185  0.8830  0.8749  ...   0.8529  0.9335  0.8981
 0.9487  0.8429  0.9129  ...   0.8104  0.9151  0.9187
 0.9108  0.8665  0.8978  ...   0.7134  0.9310  0.8733
          ...             ⋱             ...          
 0.9575  0.9496  0.9318  ...   0.8885  0.9243  0.9432
 0.7996  0.8863  0.8948  ...   0.7289  0.9377  0.8862
 0.9416  0.9361  0.9136  ...   0.8739  0.8925  0.9247
[torch.FloatTensor of size 50x21]
 is loss and max idx
Variable containing:
 0.2306
[torch.FloatTensor of size 1]
 Variable containing:
 0.8181  0.9446  0.9479  ...   0.8952  0.8288  0.9211
 0.8233  0.9057  0.9049  ...   0.8465  0.8431  0.8822
 0.8035  0.9120  0.8719  ...   0.8608  0.9231  0.8756
          ...             ⋱             ...          
 0.9313  0.9430  0.9775  ...   0.8954  0.9251  0.9436
 0.8240  0.8150  0.8075  ...   0.7941  0.8502  0.8529
 0.8632  0.9057  0.9122  ...   0.8683  0.8765  0.8982
[torch.FloatTensor of size 

Variable containing:
 0.1880
[torch.FloatTensor of size 1]
 Variable containing:
 0.8957  0.8314  0.8861  ...   0.8831  0.9285  0.8448
 0.9059  0.8677  0.9146  ...   0.8509  0.8753  0.9174
 0.8243  0.8397  0.9177  ...   0.8050  0.9200  0.8823
          ...             ⋱             ...          
 0.9160  0.9168  0.9004  ...   0.9044  0.8967  0.9253
 0.8330  0.8517  0.7485  ...   0.7416  0.8313  0.7689
 0.8909  0.8400  0.8917  ...   0.8573  0.8829  0.8718
[torch.FloatTensor of size 50x21]
 is loss and max idx
Variable containing:
 0.1827
[torch.FloatTensor of size 1]
 Variable containing:
 0.9177  0.9403  0.9076  ...   0.9353  0.9284  0.8648
 0.9185  0.9554  0.9167  ...   0.9542  0.9285  0.9252
 0.9387  0.9524  0.9023  ...   0.9407  0.9223  0.8972
          ...             ⋱             ...          
 0.9457  0.9561  0.9480  ...   0.9713  0.9619  0.9393
 0.8293  0.8772  0.8205  ...   0.9082  0.8738  0.8540
 0.8592  0.9189  0.9363  ...   0.8938  0.8967  0.9318
[torch.FloatTensor of size 

Variable containing:
 0.1796
[torch.FloatTensor of size 1]
 Variable containing:
 0.8903  0.9170  0.8102  ...   0.8627  0.8519  0.9187
 0.9176  0.9392  0.8145  ...   0.8956  0.8245  0.9211
 0.9090  0.8960  0.8053  ...   0.8537  0.8180  0.9552
          ...             ⋱             ...          
 0.9743  0.9593  0.9128  ...   0.9520  0.9140  0.9807
 0.8848  0.8524  0.7425  ...   0.8061  0.8028  0.9265
 0.9195  0.9330  0.8380  ...   0.9179  0.8867  0.9660
[torch.FloatTensor of size 50x21]
 is loss and max idx
Variable containing:
 0.1575
[torch.FloatTensor of size 1]
 Variable containing:
 0.8268  0.7819  0.8428  ...   0.8331  0.8047  0.8158
 0.8890  0.8157  0.8327  ...   0.8584  0.8209  0.8639
 0.8234  0.7946  0.7776  ...   0.7340  0.7959  0.7862
          ...             ⋱             ...          
 0.9035  0.8875  0.8857  ...   0.8771  0.8765  0.8851
 0.7989  0.7529  0.7720  ...   0.7241  0.7654  0.6995
 0.8701  0.8380  0.8513  ...   0.8756  0.8855  0.8416
[torch.FloatTensor of size 

Variable containing:
 0.1706
[torch.FloatTensor of size 1]
 Variable containing:
 0.9276  0.8361  0.8943  ...   0.9199  0.9579  0.8452
 0.9049  0.8115  0.8628  ...   0.8671  0.9141  0.8172
 0.8696  0.7954  0.8177  ...   0.8129  0.9097  0.7734
          ...             ⋱             ...          
 0.9382  0.8990  0.8720  ...   0.9130  0.9517  0.8389
 0.8407  0.8294  0.8260  ...   0.8961  0.8938  0.7270
 0.9156  0.8712  0.9172  ...   0.9353  0.9195  0.8026
[torch.FloatTensor of size 50x21]
 is loss and max idx
Variable containing:
 0.1532
[torch.FloatTensor of size 1]
 Variable containing:
 0.9227  0.9392  0.8698  ...   0.8683  0.9069  0.8666
 0.9474  0.9295  0.8720  ...   0.8350  0.7993  0.8981
 0.9211  0.9325  0.9224  ...   0.8522  0.9161  0.8811
          ...             ⋱             ...          
 0.9586  0.9368  0.9567  ...   0.9012  0.9357  0.9376
 0.9024  0.8952  0.8680  ...   0.7830  0.8303  0.8249
 0.9221  0.8975  0.9110  ...   0.8611  0.8847  0.8998
[torch.FloatTensor of size 

KeyboardInterrupt: 

In [19]:
len(train_titles_only)

22853

In [30]:
criterion(Variable(torch.FloatTensor([1,2,3,4,5,6,7,8,9,10,10.5])),Variable(torch.LongTensor([10])))

Variable containing:
 0
[torch.FloatTensor of size 1]

In [35]:
sample[0][0:2]


( 0 ,.,.) = 
 -0.0239 -0.0263  0.0928  ...   0.0000  0.0000  0.0000
  0.0184 -0.1224 -0.0513  ...   0.0000  0.0000  0.0000
  0.0470 -0.0165 -0.0597  ...   0.0000  0.0000  0.0000
           ...             ⋱             ...          
  0.0181 -0.0052  0.0548  ...   0.0000  0.0000  0.0000
 -0.0214  0.0341  0.0412  ...   0.0000  0.0000  0.0000
 -0.0869  0.0506 -0.0125  ...   0.0000  0.0000  0.0000

( 1 ,.,.) = 
 -0.0472 -0.0185  0.1273  ...   0.0000  0.0000  0.0000
  0.0009 -0.1522 -0.0653  ...   0.0000  0.0000  0.0000
  0.0159  0.0979  0.0697  ...   0.0000  0.0000  0.0000
           ...             ⋱             ...          
  0.0695 -0.0083  0.1797  ...   0.0000  0.0000  0.0000
 -0.0613 -0.0455 -0.0352  ...   0.0000  0.0000  0.0000
 -0.1591 -0.0340 -0.0792  ...   0.0000  0.0000  0.0000
[torch.FloatTensor of size 2x200x38]

In [ ]:
#need to figure out how to use all